###### Imports and Settings

In [2]:
import pandas as pd
import numpy as np
import requests
from functools import reduce
import matplotlib.pyplot as plt
import pickle
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
import sys
sys.path.append("../../Functions and Dictionaries") # Adds higher directory to python modules path
import geodict
namestocommon = geodict.namestocommon
geotogeoid = geodict.geotogeoid
tofullcensus = geodict.tofullcensus
import sqlite3 as sq

# This notebook outlines the download and formatting process for the University of Wisconsin's Population Health Institute's County Health Rankings for counties places in the GNRC operating region.  

Go to this page: https://www.countyhealthrankings.org/health-data/tennessee?year=2024  
Download the "(YEAR) Tennessee Data" excel workbook  
If the new version is the same as the 2022 download, then it will contain the following pages:  
+ Introduction, Outcomes & Factors Rankings, Outcomes & Factors SubRankings, Ranked Measure Data, Additional Measure Data, Ranked Measure Sources & Years, Additional Measure Sources & Years  
+ For both "Outcomes & Factors Rankings", "Outcomes & Factors SubRankings", and Additional Measure Data": unmerge headers and keep the FIPS, rename the County to "NAME", delete the State column, and keep only the ranks (we won't use the z-scores), and make sure the column headers are consistent with the outcomes or factors. Ensure the counties are alphabetized and move all of the rankings into one sheet.
  

Save these csvs as they come in the Data Downloads folder of Parent Data Gathering  


# I've got 2019 - 2023 formatted this way, then in 2024 they stopped ranking. This is the rankings up to 2023.


## Merge Multiple Years

In [18]:
df = pd.read_csv('../Data Downloads/CountyHealthRankings_2024.csv', dtype = str)
df.head(3)

,NAME,Year,Health Outcomes,Health Factors,Life Expectancy,% Frequent Physical Distress,% Frequent Mental Distress
0,Anderson,2024,0.56 to 0.95,-0.22 to 0,73.27910482,13.2,19.4
1,Bedford,2024,0.56 to 0.95,0.23 to 0.47,73.05758623,13.9,19.2
2,Benton,2024,0.56 to 0.95,0.47 to 0.75,71.30006552,15,21.1


In [19]:
colstt = ['Health Outcomes', 'Health Factors']

In [20]:
df.set_index(['NAME', 'Year'], inplace = True)
df = df[colstt]

In [21]:
df.head()

,,Health Outcomes,Health Factors
NAME,Year,,
Anderson,2024,0.56 to 0.95,-0.22 to 0
Bedford,2024,0.56 to 0.95,0.23 to 0.47
Benton,2024,0.56 to 0.95,0.47 to 0.75
Bledsoe,2024,0.22 to 0.56,0.75 to 1.11
Blount,2024,-0.1 to 0.21,-0.44 to -0.22


In [22]:
df['Health Outcomes'] = df['Health Outcomes'].apply(lambda x: tuple(map(float, x.split(' to '))))
df['Health Factors'] = df['Health Factors'].apply(lambda x: tuple(map(float, x.split(' to '))))

In [23]:
df.head()

,,Health Outcomes,Health Factors
NAME,Year,,
Anderson,2024,"(0.56, 0.95)","(-0.22, 0.0)"
Bedford,2024,"(0.56, 0.95)","(0.23, 0.47)"
Benton,2024,"(0.56, 0.95)","(0.47, 0.75)"
Bledsoe,2024,"(0.22, 0.56)","(0.75, 1.11)"
Blount,2024,"(-0.1, 0.21)","(-0.44, -0.22)"


This approach takes the upper and lower bounds separately

In [24]:
df['Outcomes_Lower'] = df['Health Outcomes'].apply(lambda x: x[0])
df['Outcomes_Upper'] = df['Health Outcomes'].apply(lambda x: x[1])

# Calculate percentile rankings for each bound
df['Outcomes_Lower_Pct'] = df['Outcomes_Lower'].rank(pct=True)
df['Outcomes_Upper_Pct'] = df['Outcomes_Upper'].rank(pct=True)

In [26]:
df['Factors_Lower'] = df['Health Factors'].apply(lambda x: x[0])
df['Factors_Upper'] = df['Health Factors'].apply(lambda x: x[1])

# Calculate percentile rankings for each bound
df['Factors_Lower_Pct'] = df['Factors_Lower'].rank(pct=True)
df['Factors_Upper_Pct'] = df['Factors_Upper'].rank(pct=True)

In [27]:
df.head()

,,Health Outcomes,Health Factors,Outcomes_Lower,Outcomes_Upper,Outcomes_Lower_Pct,Outcomes_Upper_Pct,Factors_Lower,Factors_Upper,Factors_Lower_Pct,Factors_Upper_Pct
NAME,Year,,,,,,,,,,
Anderson,2024,"(0.56, 0.95)","(-0.22, 0.0)",0.56,0.95,0.563158,0.563158,-0.22,0.00,0.157895,0.157895
Bedford,2024,"(0.56, 0.95)","(0.23, 0.47)",0.56,0.95,0.563158,0.563158,0.23,0.47,0.600000,0.600000
Benton,2024,"(0.56, 0.95)","(0.47, 0.75)",0.56,0.95,0.563158,0.563158,0.47,0.75,0.863158,0.863158
Bledsoe,2024,"(0.22, 0.56)","(0.75, 1.11)",0.22,0.56,0.263158,0.263158,0.75,1.11,0.968421,0.968421
Blount,2024,"(-0.1, 0.21)","(-0.44, -0.22)",-0.10,0.21,0.105263,0.105263,-0.44,-0.22,0.068421,0.068421


In [32]:
top = df.sort_values('Factors_Upper', ascending = False)
top.head(10)

,,Health Outcomes,Health Factors,Outcomes_Lower,Outcomes_Upper,Outcomes_Lower_Pct,Outcomes_Upper_Pct,Factors_Lower,Factors_Upper,Factors_Lower_Pct,Factors_Upper_Pct
NAME,Year,,,,,,,,,,
Lake,2024,"(1.42, 2.0)","(1.12, 1.87)",1.42,2.00,0.978947,0.978947,1.12,1.87,1.000000,1.000000
Clay,2024,"(0.95, 1.42)","(0.75, 1.11)",0.95,1.42,0.857895,0.857895,0.75,1.11,0.968421,0.968421
Bledsoe,2024,"(0.22, 0.56)","(0.75, 1.11)",0.22,0.56,0.263158,0.263158,0.75,1.11,0.968421,0.968421
Lauderdale,2024,"(0.95, 1.42)","(0.75, 1.11)",0.95,1.42,0.857895,0.857895,0.75,1.11,0.968421,0.968421
Hancock,2024,"(1.42, 2.0)","(0.75, 1.11)",1.42,2.00,0.978947,0.978947,0.75,1.11,0.968421,0.968421
Perry,2024,"(0.56, 0.95)","(0.75, 1.11)",0.56,0.95,0.563158,0.563158,0.75,1.11,0.968421,0.968421
Van Buren,2024,"(0.95, 1.42)","(0.47, 0.75)",0.95,1.42,0.857895,0.857895,0.47,0.75,0.863158,0.863158
Macon,2024,"(0.56, 0.95)","(0.47, 0.75)",0.56,0.95,0.563158,0.563158,0.47,0.75,0.863158,0.863158
Wayne,2024,"(0.22, 0.56)","(0.47, 0.75)",0.22,0.56,0.263158,0.263158,0.47,0.75,0.863158,0.863158


This approach assumes that all of the values in the range occur with the same frequency and we are getting the 50th percentile (median) value

In [41]:
# Function to compute the 50th percentile (median) for a range
def compute_med(range_tuple):
    start, end = range_tuple
    flattened_values = np.arange(start, end + 1)  # Generate the range
    return np.percentile(flattened_values, 50)    # Calculate the median (50th percentile)

# Apply the function to the column of tuples
df['Outcomes_Med'] = df['Health Outcomes'].apply(compute_med)
df['Factors_Med'] = df['Health Factors'].apply(compute_med)

In [42]:
df.head()

,,Health Outcomes,Health Factors,Outcomes_Lower,Outcomes_Upper,Outcomes_Lower_Pct,Outcomes_Upper_Pct,Factors_Lower,Factors_Upper,Factors_Lower_Pct,Factors_Upper_Pct,Outcomes_Med,Factors_Med
NAME,Year,,,,,,,,,,,,
Anderson,2024,"(0.56, 0.95)","(-0.22, 0.0)",0.56,0.95,0.563158,0.563158,-0.22,0.00,0.157895,0.157895,1.06,0.28
Bedford,2024,"(0.56, 0.95)","(0.23, 0.47)",0.56,0.95,0.563158,0.563158,0.23,0.47,0.600000,0.600000,1.06,0.73
Benton,2024,"(0.56, 0.95)","(0.47, 0.75)",0.56,0.95,0.563158,0.563158,0.47,0.75,0.863158,0.863158,1.06,0.97
Bledsoe,2024,"(0.22, 0.56)","(0.75, 1.11)",0.22,0.56,0.263158,0.263158,0.75,1.11,0.968421,0.968421,0.72,1.25
Blount,2024,"(-0.1, 0.21)","(-0.44, -0.22)",-0.10,0.21,0.105263,0.105263,-0.44,-0.22,0.068421,0.068421,0.40,0.06


In [44]:
top = df.sort_values('Factors_Med', ascending = False)
top.head(10)

,,Health Outcomes,Health Factors,Outcomes_Lower,Outcomes_Upper,Outcomes_Lower_Pct,Outcomes_Upper_Pct,Factors_Lower,Factors_Upper,Factors_Lower_Pct,Factors_Upper_Pct,Outcomes_Med,Factors_Med
NAME,Year,,,,,,,,,,,,
Lake,2024,"(1.42, 2.0)","(1.12, 1.87)",1.42,2.00,0.978947,0.978947,1.12,1.87,1.000000,1.000000,1.92,1.62
Clay,2024,"(0.95, 1.42)","(0.75, 1.11)",0.95,1.42,0.857895,0.857895,0.75,1.11,0.968421,0.968421,1.45,1.25
Bledsoe,2024,"(0.22, 0.56)","(0.75, 1.11)",0.22,0.56,0.263158,0.263158,0.75,1.11,0.968421,0.968421,0.72,1.25
Lauderdale,2024,"(0.95, 1.42)","(0.75, 1.11)",0.95,1.42,0.857895,0.857895,0.75,1.11,0.968421,0.968421,1.45,1.25
Hancock,2024,"(1.42, 2.0)","(0.75, 1.11)",1.42,2.00,0.978947,0.978947,0.75,1.11,0.968421,0.968421,1.92,1.25
Perry,2024,"(0.56, 0.95)","(0.75, 1.11)",0.56,0.95,0.563158,0.563158,0.75,1.11,0.968421,0.968421,1.06,1.25
Van Buren,2024,"(0.95, 1.42)","(0.47, 0.75)",0.95,1.42,0.857895,0.857895,0.47,0.75,0.863158,0.863158,1.45,0.97
Macon,2024,"(0.56, 0.95)","(0.47, 0.75)",0.56,0.95,0.563158,0.563158,0.47,0.75,0.863158,0.863158,1.06,0.97
Wayne,2024,"(0.22, 0.56)","(0.47, 0.75)",0.22,0.56,0.263158,0.263158,0.47,0.75,0.863158,0.863158,0.72,0.97


I'm going to rank the medians of the ranges and then import that into the larger file

In [67]:
data = df.reset_index(drop = False)
data = data[['NAME', 'Year', 'Outcomes_Med', 'Factors_Med']]

In [68]:
data = data.sort_values('Outcomes_Med', ascending = True).reset_index(drop = True)
data = data.reset_index(drop = False)

In [69]:
data.head()

,index,NAME,Year,Outcomes_Med,Factors_Med
0,0,Williamson,2024,-1.26,-1.12
1,1,Wilson,2024,0.10,-0.17
2,2,Sumner,2024,0.10,0.06
3,3,Rutherford,2024,0.10,0.06
4,4,Blount,2024,0.40,0.06


In [70]:
data['Health Outcomes Ranking'] = data['index'] + 1
data = data.drop(columns = 'index')

In [71]:
data.head()

,NAME,Year,Outcomes_Med,Factors_Med,Health Outcomes Ranking
0,Williamson,2024,-1.26,-1.12,1
1,Wilson,2024,0.10,-0.17,2
2,Sumner,2024,0.10,0.06,3
3,Rutherford,2024,0.10,0.06,4
4,Blount,2024,0.40,0.06,5


In [72]:
data = data.sort_values('Factors_Med', ascending = True).reset_index(drop = True)
data = data.reset_index(drop = False)

In [73]:
data.head()

,index,NAME,Year,Outcomes_Med,Factors_Med,Health Outcomes Ranking
0,0,Williamson,2024,-1.26,-1.12,1
1,1,Wilson,2024,0.10,-0.17,2
2,2,Washington,2024,0.40,0.06,14
3,3,Hamilton,2024,0.40,0.06,10
4,4,Knox,2024,0.40,0.06,9


In [74]:
data['Health Factors Ranking'] = data['index'] + 1
data = data.drop(columns = 'index')

In [75]:
data.head()

,NAME,Year,Outcomes_Med,Factors_Med,Health Outcomes Ranking,Health Factors Ranking
0,Williamson,2024,-1.26,-1.12,1,1
1,Wilson,2024,0.10,-0.17,2,2
2,Washington,2024,0.40,0.06,14,3
3,Hamilton,2024,0.40,0.06,10,4
4,Knox,2024,0.40,0.06,9,5


In [76]:
data.tail()

,NAME,Year,Outcomes_Med,Factors_Med,Health Outcomes Ranking,Health Factors Ranking
90,Bledsoe,2024,0.72,1.25,27,91
91,Lauderdale,2024,1.45,1.25,73,92
92,Perry,2024,1.06,1.25,45,93
93,Hancock,2024,1.92,1.25,94,94
94,Lake,2024,1.92,1.62,95,95


In [77]:
data = data[['NAME', 'Year', 'Health Outcomes Ranking', 'Health Factors Ranking']]

In [78]:
#export to the SQLite database
conn = sq.connect('../Outputs/CountyHealthRankings.db')
data.to_sql('UWisconsin_CountyHealthRankings_2024', conn, if_exists = 'replace', index = False)

95